In [12]:
# Import the modules we need
from IPython.core.debugger import set_trace

from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from Crypto.Hash import SHA256
import binascii
from time import time
import random
import  jdc

from collections import OrderedDict
import hashlib
import json

## 1. Wallet

Zusatzinfo: Es ist wichtig die Intuition hinter der asymetrischen Kryptografie zu verstehen. Wer sich darunter nichts vorstellen kann und in der Vorlesung nicht folgen konnte, dem helfen folgende Videos vielleicht weiter:
* [Private Key / Public Key Kryptografie allg.](https://www.youtube.com/watch?v=GSIDS_lvRv4)
* [Eliptic Curve](https://www.youtube.com/watch?v=NF1pwjL9-DE)


Wallets haben zwei wesentliche Funktionen: 
1. Guthabenverwaltung 
2. Transaktionsdurchführung

Die Basis für beide Funktionen stellt der `Private Key` dar. Dieser entspricht einfach einem zufällig erzeugten (sehr langen) Passwort in einem bestimmten Format. Vom `Private Key` kann sowohl der `Public Key` abgeleitet, als auch Transaktionen signiert werden (wird später eingeführt).

In [3]:
class Wallet:
    # Basic Constructor
    def __init__(self):
        keypair = self.generate_keypair()
        self.private_key = keypair['private_key']
        self.public_key = keypair['public_key']
    
    def generate_keypair(self):
        # Generate the private key (in here the randomness takes place)
        private_key = ECC.generate(curve='P-256')
        # We return a dictionary with private and public key
        keypair = {
        'private_key': binascii.hexlify(private_key.export_key(format='DER')).decode('utf8'),
        'public_key': binascii.hexlify(private_key.public_key().export_key(format='DER')).decode('utf8')
        }
        return keypair

In [4]:
# Testing it:

wallet_alice = Wallet()
wallet_bob = Wallet()

print("Alice's Wallet:")
print(wallet_alice.private_key)
print('\n')
print(wallet_alice.public_key)
print("Bob's Wallet:")
print(wallet_bob.private_key)
print('\n')
print(wallet_bob.public_key)

Alice's Wallet:
308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b02010104201890ebbcd1506a4db674800b87639d45b831ede0638a93fdb0c132c4056a0428a144034200044c75d5dbf799a85c0888752229a73ad15aec0bcddcd770a8c8d1e785f4cdbeec163828b09255d8d9560c526ef2d9c143f42da40e1ba8d5f6d72fa51fcb2b58b7


3059301306072a8648ce3d020106082a8648ce3d030107034200044c75d5dbf799a85c0888752229a73ad15aec0bcddcd770a8c8d1e785f4cdbeec163828b09255d8d9560c526ef2d9c143f42da40e1ba8d5f6d72fa51fcb2b58b7
Bob's Wallet:
308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b02010104204484d7d008d4f4b924203a428c39a289a35c582a7abfb78ec96386f4ab475f68a14403420004a212c875e1f4a72ea51fb9a191d193f202f3e3a59f9bf817df3e427f826133328ea7d13c55ae154643fb3dfaf390472932e0b02143b2865027a013e70180d5d9


3059301306072a8648ce3d020106082a8648ce3d03010703420004a212c875e1f4a72ea51fb9a191d193f202f3e3a59f9bf817df3e427f826133328ea7d13c55ae154643fb3dfaf390472932e0b02143b2865027a013e70180d5d9


#### Wallet zu einem bestimmten Private Key
Wir können die Klasse nun so erweitern, dass wir für den Fall eines schon existierenden `Private Key` ebenfalls eine Wallet erzeugt werden kann. Beachte, dass die `ECC.import_key()` Funktion immer einen binary Input benötigt, wir speichern die Keys aber in der Regel im hexadecimalen Format als String ab. Deshalb is die Konvertierung durch binascii.unhexlify() notwendig. D.h. der `private_key_str` wird bei uns immer im hexadecimalen Format als String dem Wallet Konstruktor übergeben.

In [5]:
class Wallet:
    def __init__(self, private_key_str = None):
        keypair = self.generate_keypair(private_key_str)
        self.private_key = keypair['private_key']
        self.public_key = keypair['public_key']
        
    
    def generate_keypair(self,private_key_str):
        # If this is a 'new' wallet, i.e. no private_key_str has been given to the constructor
        # --> Create a private key
        if private_key_str is None:
            # Generate the private key (in here the randomness takes place)
            private_key = ECC.generate(curve='P-256')
        else:
            # In case this is a wallet for a specific private_key
            # read the private key in string format.
            # binascii.unhexlify(x) converts x which is in hexadecimal format into a binary format
            private_key = ECC.import_key(binascii.unhexlify(private_key_str))

        # Returns a dictionary with private and public key
        keypair = {
        'private_key': binascii.hexlify(private_key.export_key(format='DER')).decode('utf8'),
        'public_key': binascii.hexlify(private_key.public_key().export_key(format='DER')).decode('utf8')
        }
        return keypair

In [6]:
# Testing it:

private_key_alice_str = wallet_alice.private_key

wallet_allice_2 = Wallet(private_key_alice_str)

if wallet_allice_2.private_key == wallet_alice.private_key:
    print('Both wallets belong to Alice!')
    
if wallet_allice_2.public_key == wallet_alice.public_key:
    print('The public keys of both wallets are equal.')

Both wallets belong to Alice!
The public keys of both wallets are equal.


## 2. Transaktionen

Eine Transaktion besteht fundamental aus Inputs und Outputs. Da ein Input letztlich nichts anderes als eine Referenz auf einen zuvor erzeugten Output ist, genügt es ausschließlich die `TransactionOutput` Klasse zu definieren. (Die konkrete Gestaltung der `Transaction` Klasse wird später behandelt, nachdem die `TransactionOutput` Klasse beschrieben wurde.)

#### Transaction Output
Die grundlegenden Bausteine unserer Währung sind *Transaction Outputs*. Noch nicht ausgegegebene *Transaction Outputs* (kurz auch *UTXOs*) definieren die Geldmenge im System. D.h. die Summe aller *UTXOs* zu einem Zeitpunkt entspricht der gesamten Menge an verfügbaren *Fed Coins* (kurz FC) zu diesem Zeitpunkt. Ein Transaktionsinput ist einfach eine Referenz (per ID) auf einen noch nicht ausgegebenen *Transaction Output*, d.h. Input und Output können letztlich durch die selbe Klasse modelliert werden. Wir sprechen im Folgenden immer von *Transaction Output*, haben aber im gleichzeitig im Hinterkopf, dass ein Input letztlich nur eine Referenz auf einen *Transaction Output* ist. Ein *Transaction Output* soll folgenden Informationen beinhalten:

1. Zeitstempel
2. Besitzer/Empfänger
3. Wert
4. Id

Damit ein *Transaction Output* eindeutig ist (theoretisch könnte zweimal genau der gleiche Output zum gleichen Zeitpunkt generiert werden - praktisch aber sehr unwahrscheinlich, dass dieser Fall eintritt), fügen wir noch ein Feld mit einem Zufallswert hinzu.

#### Hash eines TransactionOutputs

Zusatzinfo: Die genaue Funktionsweise des SH-Algorithmus, den wir zum hashen verdwenden, wird in diesem [video](https://www.youtube.com/watch?v=DMtFhACPnTY) erklärt.

Wir wollen, dass jeder *Transaction Output* eine eindeutige ID hat die an dessen Inhalt gekoppelt ist. D.h. wenn sich der Inhalt verändert, soll sich auch die ID verändern. Zusätzlich soll es unmöglich sein, durch die ID auf den Inhalt schließen zu können. Hierzu verwenden wir eine Hashfunktion. Die Hashfunktion benötigt als Input einen beliebig langen `String` und gibt einen Hash mit festgelegter Länge als Output zurück. Weil es nicht trivial ist ein generisches Pythonobjekt direkt in einen `String` umzuwandeln, gehen wir über den Umweg eines sogenannten OrderedDictionary's (`OrderedDict`). Aus jedem `OrderedDict` kann einfach ein String erzeugt werden (z.B. im `json` Format). D.h. um den Hash eines `TransactionOutput` Objekts zu erzeugen, müssen wir das Objekt
1. In ein `OrderedDict` umwandeln, 
2. Das `OrderedDict` in einen `String` umwandeln und 
3. Den Hashwert des `Strings` berechnen.  

Es ist sinnvoll die beiden Schritte 2 und 3 in eine Funktion auszulagern, weil wir die beiden Schritte in genau dieser Form für sämtliche Bestandteile unserer Kryptowährung (z.B. Blocks) widerverwenden können. Schritt 1 wird innerhalb der Klasse `TransactionOutput` implementiert, da sich der Schritt zwischen den einzelnen Objekttypen unserer Kryptowährung jeweils unterscheidet.

#### *Zusatzinfo: Defaultwerte der Parameter*
Die Parameter, die in der Implemetierung einer Methode mit Default-Werten initialisiert werden, müssen immer am Ende der Parameterliste stehen.

In [7]:
def hash_stuff(ordered_dict_of_stuff):
    # Creates a SHA-256 hash of a the ordered_dict_of_stuff
    
    # Create a string representation of the ordered dict 
    # (via a json object)
    stuff_string = json.dumps(ordered_dict_of_stuff).encode('utf8')
    # Return the hexadecial 256 bit hash of the string
    return hashlib.sha256(stuff_string).hexdigest()



class TransactionOutput:
    def __init__(self, recipient_public_key, value, random_val = None, id = None, timestamp = None):
        # Timestamp is none if new output is created, 
        # not none if old one is referenced/recreated
        if timestamp is None:
            self.time = time()
        else:
            self.time = timestamp
            
        # We need the random_val so that each output is guaranteed to be unique, even for same t.
        # I.e., in theory, if two TransactionOutputs (for the same recipient) 
        # are generated at the exact same time we still want to be able to distinguish them.
        if random_val is None:
            # self.random = 0
            self.random = random.randint(0,10000000000000) 
        else:
            # self.random = 0
            self.random = random_val
            
        self.recipient = recipient_public_key # This defines the owner of the coins
        self.value = value # Simply the value of the output
        
        # If it's a new transaction calculate the hash,
        # if it's an existing one read the hash from the argument given at initialization.
        if id is None:
            self.id = hash_stuff(self.odict_transaction_output())
        else:
            self.id = id
    
    # A hash function needs some String as input. 
    # To get a String from a TO object (not the entire one)
    # we use a OrderedDict representation which can be converted into a String.
    def odict_transaction_output(self):
        transaction_output_odict =  OrderedDict({
                            'time' : self.time,
                            'random' : self.random,
                            'recipient' : self.recipient,
                            'value': self.value
                            })
        return(transaction_output_odict)

    def get_full_odict(self):
        # We need this to get a full OrderedDict (incl. ID)
        # representation of the TO in order to enable Transaction
        # hashing on the Transaction level later on.
        response = self.odict_transaction_output()
        response['id'] = self.id
        return response


In [8]:
# Testing it:
# If you run this code multiple times, the output below should always change
# Try setting self.random = 0 in the constructor and check if it still does 
# (it should due to the timestamp)
transaction_output_to_alice = TransactionOutput(wallet_alice.public_key, 10)
print(transaction_output_to_alice.id)

b4393e7315463c4000a2cac15025ce6fce8db6ebf86574b570c588886867d4bd


In [9]:
print(transaction_output_to_alice.get_full_odict())

OrderedDict([('time', 1540485863.886085), ('random', 6108072526798), ('recipient', '3059301306072a8648ce3d020106082a8648ce3d030107034200044c75d5dbf799a85c0888752229a73ad15aec0bcddcd770a8c8d1e785f4cdbeec163828b09255d8d9560c526ef2d9c143f42da40e1ba8d5f6d72fa51fcb2b58b7'), ('value', 10), ('id', 'b4393e7315463c4000a2cac15025ce6fce8db6ebf86574b570c588886867d4bd')])


#### Transaction

Zusätzlich zu den fundamentalen Bestandteilen, den Inputs und Outputs, wird in einer Transaktion auch noch die Idendität des Senders und eine Signatur gespeichert. Damit eine Transaktion gültig ist, muss die Summe der Inputs gleich der Summe der Outputs sein. Das bedeutet, wenn Bob eine Banane von Alice für 5 FC kauft und dies mit einem Input von 10 FC bezahlen will, so muss er für die Gültigkeit einer Transaktion die restlichen 5 FC wieder an sich selbst zurücküberweisen. 
<br/>Zusätzlich wird für die Gültigkeit einer Transaktion gefordert, dass alle Inputs zum Zeitpunkt der Aufnahmen in die Blockchain im Besitz des Senders sind und die Signatur mit der Idendität des Senders übereinstimmt.
Da die Besitzverhältnisse der FC in der Blockchain erfasst werden, wir der die Besitzverhältnisse referenzierende Teil erst in der nächsten Veranstaltung ergänzt, da in dieser Veranstaltung dann auch die Blockchain eingeführt wird.


In [10]:
class Transaction:
    def __init__(self, sender_pub_key, inputs, outputs, timestamp = None ,signature = None, transaction_id = None):
        # Note: This is somehow smilar to the initialization
        # of TransactionOutput, except of the fact that there 
        # are different 'fields' in the Transaction class.
        if timestamp is None:
            self.timestamp = time() #include, to not have a hash collision for similar transactions
        else:
            self.timestamp = timestamp
        self.sender = sender_pub_key
        self.inputs = inputs # list of IDs referring to UTXOs
        self.outputs = outputs # OrderedDict with id:TransactionOutput
        
        # Signature and ID are (re)created when the transaction is signed
        # Note: The coinbase transaction has no signature
        if signature is None:
            self.signature = ''
        else:
            self.signature = signature

        if transaction_id is None:
            self.id = self.hash_transaction()
        else:
            self.id = transaction_id

    def odict_transaction(self):
        # Create an OrderedDict representation
        # We need this to sign it later on, therefore the 
        # signature is not included in the OrderedDict.
    
        outputs_odict = OrderedDict()
        for output in self.outputs.values():
            outputs_odict[output.id]=output.get_full_odict()
        transaction_odict =  OrderedDict({
                            'timestamp' : self.timestamp,
                            'sender': self.sender,
                            'inputs': self.inputs,
                            'outputs': outputs_odict})
        return(transaction_odict)

    def hash_transaction(self):
        # Creates a hash of the transaction
        # We need this extra function since we have to
        # add the signature to the OrderedDict object
        # manually. Otherwise we could simply use the hash_stuff()
        # from above directly.
        
        transaction_dict = self.odict_transaction()
        transaction_dict['signature'] = self.signature
        transaction_string = json.dumps(transaction_dict).encode()
        return hashlib.sha256(transaction_string).hexdigest()

    
    def get_full_odict(self):
        # We need this to get a full OrderedDict (incl. ID)
        # rerepsentation of a Transaction to enable
        # hashing of blocks on the block-level later on.
        
        response = self.odict_transaction()
        response['signature'] = self.signature
        response['id'] = self.id
        return response
    


In [11]:
# Testing the initialization part:
# Idea: send coins from Alice to Bob, i.e., Alice buys a coffee 
# from Bob for 5 FC.
# 1. We use the TO belonging to Alice from the 
# `transaction_output_to_alice` from above.
# 2. We create two TOs with each 5 FC, one belonging to Bob
# (price of the coffee) and the other one
# belonging to Alice (change). 
# 3. We create an OrderedDict of theses two TOs with id:TO,
# since this is the required input format for the transaction.
# 4. We create the transaction.

# Step 1.
# Use `transaction_output_to_alice` created in previous test

# Step 2
transaction_output_to_bob = TransactionOutput(wallet_bob.public_key, 5)
transaction_output_to_alice_2 = TransactionOutput(wallet_alice.public_key, 5)

# Step 3.
outputs_od = OrderedDict({transaction_output_to_bob.id:transaction_output_to_bob,
                        transaction_output_to_alice_2.id: transaction_output_to_alice_2})

# Step 4.
transaction_alice_to_bob = Transaction(wallet_alice.public_key,
                                       [transaction_output_to_alice.id],
                                       outputs_od)

print(transaction_alice_to_bob.get_full_odict())

OrderedDict([('timestamp', 1540485885.888886), ('sender', '3059301306072a8648ce3d020106082a8648ce3d030107034200044c75d5dbf799a85c0888752229a73ad15aec0bcddcd770a8c8d1e785f4cdbeec163828b09255d8d9560c526ef2d9c143f42da40e1ba8d5f6d72fa51fcb2b58b7'), ('inputs', ['b4393e7315463c4000a2cac15025ce6fce8db6ebf86574b570c588886867d4bd']), ('outputs', OrderedDict([('15dba39705775f79928acbc65c1d956c604a9ab3e7fc879aaaa6b354751f5b4c', OrderedDict([('time', 1540485885.888719), ('random', 2440267349715), ('recipient', '3059301306072a8648ce3d020106082a8648ce3d03010703420004a212c875e1f4a72ea51fb9a191d193f202f3e3a59f9bf817df3e427f826133328ea7d13c55ae154643fb3dfaf390472932e0b02143b2865027a013e70180d5d9'), ('value', 5), ('id', '15dba39705775f79928acbc65c1d956c604a9ab3e7fc879aaaa6b354751f5b4c')])), ('030f17f69b0a7daeabf6346e09fa81369cfffc82406f1d57ab647dd6a3bf4c9b', OrderedDict([('time', 1540485885.8887959), ('random', 9225901234913), ('recipient', '3059301306072a8648ce3d020106082a8648ce3d030107034200044c75d5db

In [16]:
%%add_to Transaction

def sign_transaction(self, private_key):
    # Sign a transaction using the private_key
        
    private_key = ECC.import_key(binascii.unhexlify(private_key))
    signer = DSS.new(private_key,'fips-186-3')
    h = SHA256.new(str(self.odict_transaction()).encode('utf8'))
    self.signature = binascii.hexlify(signer.sign(h)).decode('utf8')
    # When the signature is created the id can be set
    self.id = self.hash_transaction()
    return self.signature
    
def verify_transaction_signature(self):
    # Check if the transaction signature is valid,
    # i.e. does the signature match the Transaction object.
    # Note: This might not make too much sense given the way we
    # create transactions in this tutorial.
    # But in case we get transaction data
    # from a remote point, this is important!
        
    public_key = ECC.import_key(binascii.unhexlify(self.sender))
    verifier = DSS.new(public_key,'fips-186-3')
    h = SHA256.new(str(self.odict_transaction()).encode('utf8'))
    try:
        verifier.verify(h, binascii.unhexlify(self.signature))
        return(True)
    # In case the signature is not authentic, the verifier throws a ValueError
    except ValueError:
        return(False)

def verify_transaction_id(self):
    # Check if the id matched the Transaction content
     
    if self.id == self.hash_transaction():
        return True

    return False

In [17]:
# Testing the signature part (1):
# Signing the Transaction using the right private_key (the one that coresponds
# the sender's public key).
# Note that the id of the Transaction changes as expected.

print(transaction_alice_to_bob.id)
transaction_alice_to_bob.sign_transaction(wallet_alice.private_key)
print(transaction_alice_to_bob.id)
# Verify the signature
print(transaction_alice_to_bob.verify_transaction_signature())

98ed0bc829ce5f00521b74ae46b15ea3929cfddb1407a52124a790b9bca8a67e
adfdd1e06c8c7df9b74acc77d16230f10c2e398fd24966dc0fc6b934b887ac3b
True


In [18]:
# Testing the signature part (2):
# Signing the Transaction using a wrong private_key.

print(transaction_alice_to_bob.id)
transaction_alice_to_bob.sign_transaction(wallet_bob.private_key)
print(transaction_alice_to_bob.id)
# Verify the signature --> This time it should return 'False'
print(transaction_alice_to_bob.verify_transaction_signature())

adfdd1e06c8c7df9b74acc77d16230f10c2e398fd24966dc0fc6b934b887ac3b
bba44c51b2359ed175dcaae7e3c35bf20f86da10a1e6b5b89a6f99cdbd316b1c
False


In [19]:
# Testing the id part (1):
# Sign it with the wrong signature 
# to get a wrong signature (for test reasons)
transaction_alice_to_bob.sign_transaction(wallet_bob.private_key)
id_with_wrong_signature = transaction_alice_to_bob.id

# Sign it with the right signature and test the id (actually trivial):
transaction_alice_to_bob.sign_transaction(wallet_alice.private_key)
print(transaction_alice_to_bob.verify_transaction_id())

True


In [20]:
# Testing the id part (2):
# Now assign the wrong id and test again:
transaction_alice_to_bob.id = id_with_wrong_signature
print(transaction_alice_to_bob.verify_transaction_id())

False


## *Zusatz: Auslagerung abgeschlossener Programmteile*

Um den Überblick über den Code nicht zu verlieren, macht es in vielen Fällen Sinn, mehr oder weniger abgeschlossene Abschnitte des Codes in eigene \*.py Dateien auszulagern. Kleinere Änderungen am Code können auch in diesen Dateien durchgeführt werden. Um Programmcode zu testen, können wir auf die Klassen und Methoden in den ausgelagerten Dateien auch wieder über die jupyter Notebooks zugreifen. Bzgl. der Implementierung von FEDCoin gehen wir hierzu folgendermaßen vor:

1. Erstelle mit einem Texteditor deiner Wahl (Sublime wird empfohlen) eine Datei `FEDCoin.py` im selben Verzeichnis wie dieses Notebook und kopiere dort zuerst die Importe aus der obersten Zelle dieses Notebooks hinein und füge zusätzlich noch folgenden Import hinzu: `from utils import *`.  Kopiere dann die jeweils fotgeschrittensten Implementierungen der Klassen `Wallet`, `TransactionOutput` und `Transaction` in diese Datei hinein.

2. Erstelle eine zweite Datei mit dem Namen `utils.py` die zunächst nur die Funktion `hash_shuff()` und die für die Funktion notwendigen Importe (`hashlib`, `json`) beinhaltet.

3. Importiere den Inhalt beider Dateien in ein jupyter Notebook und erzeuge z.B. ein neues Wallet damit. Um sämtliche Inhalte aus einer \*.py Datei zu importieren, verwendet man einen Befehl à la `from FEDCoin import *` (siehe Beispiel unten).


In [21]:
# Testing the import of code from *.py files in the same folder:
# Note: To make sure that you don't use the cached classes
# go to the menue bar of this notebook and select
# --->Kernel--->Restart & Clear Output
# Then run the code in this cell.

from FEDCoin import *
from utils import *

wallet_peter = Wallet()
print(wallet_peter.public_key)
print("\n")

test_od = OrderedDict({'bla':'blub'})
print(hash_stuff(test_od))

3059301306072a8648ce3d020106082a8648ce3d03010703420004fe746e709b2f19952cf01209bd11ef0873da549145b58a4de96c8aac9826d596c97501dc593130a0ea7b2131c316367b291d5dc1f40ef84584cc365b3cc709aa


80ce0e90c0b8d1a4e42e997a073c894b4d7a4cedc619885ec7a3ea8dbc282178
